**Title:** Lab Assignment 2: Sentiment Classification with Machine Learning

**Author:** Aditi Krishna Priya Manikantan

**ASU ID:** 1230651112

**Date:** January 20, 2023

In [2]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.3 MB/s eta 0:00:00


**Code Cell 1 - Library and data import, using the first 10000 rows.**

In [38]:
# Import necessary libraries
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.tokenize import RegexpTokenizer
import json
import spacy

# Load the dataset
data = pd.read_csv('restaurant_reviews_az.csv', nrows=10000)
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,Sentiment
0,IVS7do_HBzroiCiymNdxDg,fdFgZQQYQJeEAshH4lxSfQ,sGy67CpJctjeCWClWqonjA,3,1,1,0,"OK, the hype about having Hatch chili in your ...",1/27/2020 22:59,1
1,QP2pSzSqpJTMWOCuUuyXkQ,JBLWSXBTKFvJYYiM-FnCOQ,3w7NRntdQ9h0KwDsksIt5Q,5,1,1,1,Pandemic pit stop to have an ice cream.... onl...,4/19/2020 5:33,1
2,oK0cGYStgDOusZKz9B1qug,2_9fKnXChUjC5xArfF8BLg,OMnPtRGmbY8qH_wIILfYKA,5,1,0,0,I was lucky enough to go to the soft opening a...,2/29/2020 19:43,1
3,E_ABvFCNVLbfOgRg3Pv1KQ,9MExTQ76GSKhxSWnTS901g,V9XlikTxq0My4gE8LULsjw,5,0,0,0,I've gone to claim Jumpers all over the US and...,3/14/2020 21:47,1
4,Rd222CrrnXkXukR2iWj69g,LPxuausjvDN88uPr-Q4cQA,CA5BOxKRDPGJgdUQ8OUOpw,4,1,0,0,"If you haven't been to Maynard's kitchen, it'...",1/17/2020 20:32,1


**Code Cell 2 - Apply lexicon-based approach with VaderSentiment to predict sentiment on the above offered Input 1 data.**

i) Associated sentiment scores

In [52]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to calculate sentiment score
def sentiment_score(review):
    print(analyzer.polarity_scores(review))
    return analyzer.polarity_scores(review)['compound']

# Apply sentiment analysis
data['vader_sentiment'] = data['text'].apply(sentiment_score)

Streaming output truncated to the last 5000 lines.
{'neg': 0.0, 'neu': 0.645, 'pos': 0.355, 'compound': 0.8539}
{'neg': 0.0, 'neu': 0.866, 'pos': 0.134, 'compound': 0.5719}
{'neg': 0.016, 'neu': 0.834, 'pos': 0.149, 'compound': 0.9802}
{'neg': 0.096, 'neu': 0.831, 'pos': 0.073, 'compound': -0.2406}
{'neg': 0.053, 'neu': 0.693, 'pos': 0.255, 'compound': 0.807}
{'neg': 0.097, 'neu': 0.706, 'pos': 0.197, 'compound': 0.8373}
{'neg': 0.033, 'neu': 0.817, 'pos': 0.15, 'compound': 0.8449}
{'neg': 0.057, 'neu': 0.753, 'pos': 0.191, 'compound': 0.7169}
{'neg': 0.167, 'neu': 0.526, 'pos': 0.308, 'compound': 0.7028}
{'neg': 0.0, 'neu': 0.674, 'pos': 0.326, 'compound': 0.9272}
{'neg': 0.061, 'neu': 0.765, 'pos': 0.175, 'compound': 0.8807}
{'neg': 0.0, 'neu': 0.91, 'pos': 0.09, 'compound': 0.765}
{'neg': 0.03, 'neu': 0.653, 'pos': 0.317, 'compound': 0.9922}
{'neg': 0.022, 'neu': 0.839, 'pos': 0.139, 'compound': 0.8652}
{'neg': 0.0, 'neu': 0.687, 'pos': 0.313, 'compound': 0.9232}
{'neg': 0.139, 'neu

ii) Model Performance

In [57]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to calculate sentiment score
def sentiment_score(review):
    # Calculate the polarity scores
    scores = analyzer.polarity_scores(review)
    # Return the compound sentiment score
    return scores['compound']

# Apply sentiment analysis to the 'text' column of the DataFrame
data['vader_sentiment'] = data['text'].apply(sentiment_score)

# Predict sentiment for each text in X_test
v_predicted = []
for text in X_test:
    sent = analyzer.polarity_scores(text)
    if sent['compound'] > 0:
        v_predicted.append(1)
    else:
        v_predicted.append(0)

# Evaluate the performance of the VADER sentiment analysis
v_performance = metrics.classification_report(Y_test, v_predicted, target_names=['0', '1'])
print(v_performance)

              precision    recall  f1-score   support

           0       0.89      0.51      0.65       577
           1       0.83      0.97      0.90      1423

    accuracy                           0.84      2000
   macro avg       0.86      0.74      0.77      2000
weighted avg       0.85      0.84      0.83      2000



**Code Cell 3 - Set up review text and sentiment label, and prepare the training and
test sets on Input 1 data for machine learning classifications, using 5-fold cross-
validation. Conduct necessary data processing**

i) Splitting into training and testing

In [54]:
#Splitting the data into trainig and testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data['text'], data['Sentiment'], test_size=0.2, random_state=5)

ii) Data Processing using Bag of Words and 5-fold Validation




In [55]:
# Initialize the tokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')

# Initialize CountVectorizer with the custom tokenizer
cv = CountVectorizer(stop_words='english', ngram_range=(1,1), tokenizer=token.tokenize, max_features=1000)

# Vectorize the text data of the training set
X_train_vect = cv.fit_transform(X_train)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [15]:
cv.vocabulary_

{'time': 884,
 'dining': 234,
 'beautiful': 80,
 'patio': 629,
 'new': 570,
 'perfect': 636,
 'way': 961,
 'spend': 818,
 'evening': 290,
 'did': 229,
 'mention': 534,
 'food': 330,
 'good': 362,
 'return': 722,
 'soon': 810,
 's': 739,
 'busy': 121,
 'place': 654,
 'went': 969,
 'locations': 492,
 'day': 214,
 'pretty': 683,
 'looking': 498,
 'bacon': 65,
 'location': 491,
 'change': 142,
 'pastries': 627,
 'didn': 230,
 't': 853,
 'like': 477,
 'selection': 771,
 'drove': 259,
 'called': 127,
 'heard': 397,
 'walked': 950,
 'people': 633,
 'waiting': 947,
 'seated': 767,
 'covid': 193,
 'wanting': 955,
 'just': 448,
 'order': 603,
 'ready': 701,
 'help': 399,
 'looked': 497,
 'packed': 616,
 'wanted': 954,
 'checked': 149,
 'visit': 940,
 'fast': 312,
 'inside': 435,
 'clean': 165,
 'staff': 827,
 'nice': 571,
 'saw': 759,
 'lost': 500,
 'lol': 493,
 'offered': 585,
 'liked': 478,
 'taste': 865,
 'amazing': 34,
 'got': 363,
 'strawberry': 839,
 'know': 456,
 'thing': 879,
 'try': 912

In [56]:
X_test_vect= cv.transform(X_test)
X_test_vect.shape

(2000, 1000)

**Code Cell 4 - Describe the processed data**

In [17]:
# variables to store term statistics
num_of_comments = 0
unique_word = set() # using the set-type variable since it does not allow duplicates > able to count the number of unique words
num_of_token_per_comment = [] # using the list-type varailbe since we want to measure corpus-level statistics (e.g., avg, max, min, median, etc.)
num_of_token_per_comment_without_stop_words = []
total_number_of_tokens = 0 # in a corpus
unique_user = set() # using the set-type variable since it does not allow duplicates > able to count the number of unique users
date_list = [] # able to measure the number of comments by day, week, etc.
vote_count = 0
unique_submission = set() # using the set-type variable since it does not allow duplicates > able to count the number of unique submissions

In [21]:
!python -m spacy download en_core_web_lg

2024-01-20 21:40:05.406379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-20 21:40:05.406464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-20 21:40:05.408408: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-20 21:40:07.248434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 986.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [25]:
# load the Spacy language model
nlp = spacy.load("en_core_web_lg")
for index, row in data.iterrows():
    text = row["text"]
    doc = nlp(text)
    num_of_comments += 1

    # statistics regarding words
    num_of_tokens = len(doc)
    total_number_of_tokens += num_of_tokens
    token_count_without_stop_words = 0

    for token in doc:
        if token.is_stop is True:
            pass
        else:
            unique_word.add(str(token).lower())
            token_count_without_stop_words += 1

    num_of_token_per_comment.append(num_of_tokens)
    num_of_token_per_comment_without_stop_words.append(token_count_without_stop_words)

DATA STATISTICS:

In [26]:
# statistics
print("number of comments:", num_of_comments)
print("number of unique words:", len(unique_word))
print("total number of words in the corpus:", total_number_of_tokens)
print("average number of words in comments:", np.mean(np.asarray(num_of_token_per_comment)))
print("average number of words in comments without stop words:", np.mean(np.asarray(num_of_token_per_comment_without_stop_words)))
print("maximum number of words in comments:", np.max(np.asarray(num_of_token_per_comment)))
print("maximum number of words in comments without stop words:", np.max(np.asarray(num_of_token_per_comment_without_stop_words)))
print("minimum number of words in comments:", np.min(np.asarray(num_of_token_per_comment)))
print("minimum number of words in comments without stop words:", np.min(np.asarray(num_of_token_per_comment_without_stop_words)))
print("median number of words in comments:", np.median(np.asarray(num_of_token_per_comment)))
print("median number of words in comments without stop words:", np.median(np.asarray(num_of_token_per_comment_without_stop_words)))
#print("number of unique users:", len(unique_user))
#print("number of sumbissions:", len(unique_submission))

number of comments: 10301
number of unique words: 18907
total number of words in the corpus: 1037459
average number of words in comments: 100.7143966605184
average number of words in comments without stop words: 51.456945927579845
maximum number of words in comments: 1108
maximum number of words in comments without stop words: 604
minimum number of words in comments: 4
minimum number of words in comments without stop words: 4
median number of words in comments: 72.0
median number of words in comments without stop words: 37.0


**Code Cells 5 - Apply naïve Bayes classification to train on Input 1 data. Report on
performance of the model.**

In [29]:
from sklearn import metrics
# Naïve Bayes classifier
classifier = MultinomialNB()
classifier.fit(X_train_vect, Y_train)

# Evaluate the classifier using cross-validation
cross_val_scores = cross_val_score(classifier, X_train_vect, Y_train, cv=kf)

# Print the accuracy for each fold
print("Accuracy for each fold: ", cross_val_scores)
#Evaluate the performance of the model
predicted = classifier.predict(X_test_vect)
performance = metrics.classification_report(Y_test,predicted, target_names= ['0', '1'])
print(performance)


Accuracy for each fold:  [0.89625  0.884375 0.885625 0.871875 0.900625]
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       577
           1       0.91      0.93      0.92      1423

    accuracy                           0.89      2000
   macro avg       0.87      0.86      0.86      2000
weighted avg       0.89      0.89      0.89      2000



**Code Cell 6 - Apply support vector machines (SVM) classification to train on Input
1 data. Report on performance of the model.**

In [41]:
clf = SVC()
clf.fit(X_train_vect, Y_train)
cross_val_scores = cross_val_score(clf, X_train_vect, Y_train, cv=kf)
# Print the accuracy for each fold
print("Accuracy for each fold: ", cross_val_scores)
predicted = clf.predict(X_test_vect)
performance = metrics.classification_report(Y_test,predicted, target_names= ['0', '1'])
print(performance)

Accuracy for each fold:  [0.890625 0.880625 0.89625  0.8875   0.90125 ]
              precision    recall  f1-score   support

           0       0.86      0.76      0.81       577
           1       0.91      0.95      0.93      1423

    accuracy                           0.90      2000
   macro avg       0.88      0.86      0.87      2000
weighted avg       0.89      0.90      0.89      2000



**Code Cell 7 - Perform TF-IDF quantification on the processed Input 1 data.**

In [32]:
text_review = data['text']  # store all reviews here so that we can fit the vectorizer with this dataset
Y = data['Sentiment']
print(text_review)
print(Y)

0       OK, the hype about having Hatch chili in your ...
1       Pandemic pit stop to have an ice cream.... onl...
2       I was lucky enough to go to the soft opening a...
3       I've gone to claim Jumpers all over the US and...
4       If you haven't been  to Maynard's kitchen, it'...
                              ...                        
9995    Found this tepanyaki restaurant a few years ag...
9996    This was our first time in Tucson. We decided ...
9997    We had a birthday dinner reservation at 8:00 w...
9998    Took my parents to dinner here for my Dads bir...
9999    Sad place this sbarro at the outlet mall. They...
Name: text, Length: 10000, dtype: object
0       1
1       1
2       1
3       1
4       1
       ..
9995    0
9996    1
9997    0
9998    1
9999    0
Name: Sentiment, Length: 10000, dtype: int64


In [33]:
vectorizer = TfidfVectorizer(max_features=1000) # the value of max_features is dependent on how you design your model
#vectorizer = TfidfVectorizer()
vectorizer.fit(text_review) # fit the vectorizer with the entire dataset
# once we fitted the vectorizer, we use it for converting raw text (review) into vectors
vectorized_text = vectorizer.transform(text_review)

print(vectorized_text.shape)
print(type(vectorized_text))
print(len(Y))
print(len(text_review))

(10000, 1000)
<class 'scipy.sparse._csr.csr_matrix'>
10000
10000


In [43]:
X_train_vect, Y_train = vectorized_text.toarray(), np.asarray(Y) # convert both input (vectorized_review) and output (y) into numpy variables
print(Y_train)

[1 1 1 ... 0 1 0]


**Code Cell 8 - Apply logistic regression to train on Input 1 data. Report on
performance of the model.**

In [44]:
LG = LogisticRegression()
#LG = LogisticRegression(max_iter=1000, solver='lbfgs')
print(X_train_vect.shape)
LG.fit(X_train_vect, Y_train)
cross_val_scores = cross_val_score(LG, X_train_vect, Y_train, cv=kf)
# Print the accuracy for each fold
print("Accuracy for each fold: ", cross_val_scores)
predicted = LG.predict(X_test_vect)
performance = metrics.classification_report(Y_test,predicted, target_names= ['0', '1'])
print(performance)

(10000, 1000)
Accuracy for each fold:  [0.9    0.901  0.8975 0.9105 0.9035]
              precision    recall  f1-score   support

           0       0.39      0.47      0.43       577
           1       0.77      0.71      0.74      1423

    accuracy                           0.64      2000
   macro avg       0.58      0.59      0.58      2000
weighted avg       0.66      0.64      0.65      2000



**Text Cell 9 - Compare and conclude your observations on the performance of
lexicon-based sentiment analysis approach and different machine learning models.**

**Overall Comparison:**

*   Machine learning models, especially Naive Bayes and SVM, generally showed a more balanced performance across both sentiment classes compared to VADER. They are likely more sensitive to the context and subtleties in the text due to their training on specific datasets.

*   VADER, being a lexicon-based method, is very effective in identifying explicit sentiment expressions but may lack the nuanced understanding of context and varied expressions that machine learning models can capture.


*   While VADER excels in straightforward sentiment identification, especially for positive sentiments, it may not be as effective in capturing the nuances and varied expressions of sentiment as machine learning models.







**Code Cell 10 - Apply the trained logistic regression model to predict sentiment on
the three customer reviews listed in the above Input 2.**

Review 1

In [48]:
new_data = ["The service is good, but location is hard to find. Sanitation is not very good with old facilities. Food served tasted extremely fishy, making us difficult to even finish it."]

# Assuming 'vectorizer' is already fitted with your training data (e.g., CountVectorizer or TfidfVectorizer)
vectorized_new_data = vectorizer.transform(new_data)

# Assuming 'LG' is your trained Logistic Regression model
# Get the probability estimates for each class
predict_scores = LG.predict_proba(vectorized_new_data)
print("Prediction Scores:\n", predict_scores)

# If you want to get the final class predictions
predictions = LG.predict(vectorized_new_data)
print("Predictions:\n", predictions)


Prediction Scores:
 [[0.73202418 0.26797582]]
Predictions:
 [0]


Review 2

In [49]:
new_data = ["The restaurant is definitely one of my favorites and of my family as well. I was especially impressed with my visit a few days ago. The place is clean, and you just need to wait for fewer than 10 minutes to get food served. And of course, the food is absolutely delicious!"]
# Assuming 'vectorizer' is already fitted with your training data (e.g., CountVectorizer or TfidfVectorizer)
vectorized_new_data = vectorizer.transform(new_data)

# Assuming 'LG' is your trained Logistic Regression model
# Get the probability estimates for each class
predict_scores = LG.predict_proba(vectorized_new_data)
print("Prediction Scores:\n", predict_scores)

# If you want to get the final class predictions
predictions = LG.predict(vectorized_new_data)
print("Predictions:\n", predictions)

Prediction Scores:
 [[0.04818284 0.95181716]]
Predictions:
 [1]


Review 3

In [50]:
new_data = ["I appreciated the friendly staff. The food was good, not amazing. The service was not prompt but almost acceptable. A reliable spot for a regular meal, but nothing extraordinary."]
# Assuming 'vectorizer' is already fitted with your training data (e.g., CountVectorizer or TfidfVectorizer)
vectorized_new_data = vectorizer.transform(new_data)

# Assuming 'LG' is your trained Logistic Regression model
# Get the probability estimates for each class
predict_scores = LG.predict_proba(vectorized_new_data)
print("Prediction Scores:\n", predict_scores)

# If you want to get the final class predictions
predictions = LG.predict(vectorized_new_data)
print("Predictions:\n", predictions)

Prediction Scores:
 [[0.46949513 0.53050487]]
Predictions:
 [1]


**Text Cell 11 (5%) - Classify these reviews in Input 2 into positive, neutral, or negative
sentiments. Include your reasoning of classification (hint: it should be based on the probability that your model produces for each review). Comment on the classification
results for this new task.**


In [51]:
# Prediction scores for each review
scores_review_1 = [0.73202418, 0.26797582]  # Input 1
scores_review_2 = [0.04818284, 0.95181716]  # Input 2
scores_review_3 = [0.46949513, 0.53050487]  # Input 3

# Function to classify based on scores
def classify_review(scores):
    negative_prob, positive_prob = scores
    threshold = 0.5  # Threshold for determining sentiment
    if abs(positive_prob - negative_prob) < 0.1:  # Adjust this margin as per your requirement for neutrality
        return "Neutral"
    elif positive_prob > negative_prob:
        return "Positive"
    else:
        return "Negative"

# Classify each review
classification_1 = classify_review(scores_review_1)
classification_2 = classify_review(scores_review_2)
classification_3 = classify_review(scores_review_3)

print("Classification for Review 1:", classification_1)
print("Classification for Review 2:", classification_2)
print("Classification for Review 3:", classification_3)


Classification for Review 1: Negative
Classification for Review 2: Positive
Classification for Review 3: Neutral



 *Based on the previous classifications, review 1 was truly negative and it was captured correctly. Review 2 is positive and it was determined perfectly. Review 3 was not extremely positiv or negative. It had a neutral which was also predicted properly*

**Text Cell 12 (5%) - Acknowledge if you have used any GenAI tools in this assignment and
anyone you have worked together with on this assignment.**

I have used GenAI tools for this assignment but I have not worked with my peers for the same.